# English🏴󠁧󠁢󠁥󠁮󠁧󠁿 to Spanish🇪🇸 Translation Model

In [37]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64          # Batch size for training
epochs = 100             # Number of epochs to train for
latent_dim = 256         # Latent dimensionality of the encoding space
num_samples = 10000      # Number of samples to train on
# Path to the input text data
data_path = '../input/eng-to-spa/spa-eng/spa.txt'

In [38]:
# Vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding = 'utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text,_ = line.split('\t')
    # We use 'tab' as the start sequence character and '\n' as the end sequence character, for the target characters
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [39]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of Samples:', len(input_texts))
print('Number of unique input Tokens:', num_encoder_tokens)
print('Number of unique output Tokens:', num_decoder_tokens)
print('Maximum sequence length for inputs:', max_encoder_seq_length)
print('Maximum sequence length for outputs:', max_decoder_seq_length)

Number of Samples: 10000
Number of unique input Tokens: 69
Number of unique output Tokens: 84
Maximum sequence length for inputs: 16
Maximum sequence length for outputs: 42


In [40]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [41]:
input_token_index

{' ': 0,
 '!': 1,
 '$': 2,
 "'": 3,
 ',': 4,
 '-': 5,
 '.': 6,
 '0': 7,
 '1': 8,
 '2': 9,
 '3': 10,
 '4': 11,
 '5': 12,
 '6': 13,
 '7': 14,
 '8': 15,
 '9': 16,
 ':': 17,
 '?': 18,
 'A': 19,
 'B': 20,
 'C': 21,
 'D': 22,
 'E': 23,
 'F': 24,
 'G': 25,
 'H': 26,
 'I': 27,
 'J': 28,
 'K': 29,
 'L': 30,
 'M': 31,
 'N': 32,
 'O': 33,
 'P': 34,
 'Q': 35,
 'R': 36,
 'S': 37,
 'T': 38,
 'U': 39,
 'V': 40,
 'W': 41,
 'Y': 42,
 'a': 43,
 'b': 44,
 'c': 45,
 'd': 46,
 'e': 47,
 'f': 48,
 'g': 49,
 'h': 50,
 'i': 51,
 'j': 52,
 'k': 53,
 'l': 54,
 'm': 55,
 'n': 56,
 'o': 57,
 'p': 58,
 'q': 59,
 'r': 60,
 's': 61,
 't': 62,
 'u': 63,
 'v': 64,
 'w': 65,
 'x': 66,
 'y': 67,
 'z': 68}

In [42]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '0': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 '4': 13,
 '5': 14,
 '6': 15,
 '7': 16,
 '8': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'L': 30,
 'M': 31,
 'N': 32,
 'O': 33,
 'P': 34,
 'Q': 35,
 'R': 36,
 'S': 37,
 'T': 38,
 'U': 39,
 'V': 40,
 'W': 41,
 'Y': 42,
 'a': 43,
 'b': 44,
 'c': 45,
 'd': 46,
 'e': 47,
 'f': 48,
 'g': 49,
 'h': 50,
 'i': 51,
 'j': 52,
 'k': 53,
 'l': 54,
 'm': 55,
 'n': 56,
 'o': 57,
 'p': 58,
 'q': 59,
 'r': 60,
 's': 61,
 't': 62,
 'u': 63,
 'v': 64,
 'w': 65,
 'x': 66,
 'y': 67,
 'z': 68,
 '¡': 69,
 '«': 70,
 '»': 71,
 '¿': 72,
 'Á': 73,
 'É': 74,
 'Ó': 75,
 'Ú': 76,
 'á': 77,
 'é': 78,
 'í': 79,
 'ñ': 80,
 'ó': 81,
 'ú': 82,
 'ü': 83}

In [43]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype = 'float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')

In [44]:
input_text, target_text

("We're losing it.", '\tLe estamos perdiendo.\n')

In [45]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1
    
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by 1 time-step
        decoder_input_data[i, t, target_token_index[char]] = 1
        if t > 0:
            # decoder_target_data will be ahead by 1 time-step and will not include the start character
            decoder_target_data[i, t - 1, target_token_index[char]] = 1
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1
    decoder_target_data[i, t:, target_token_index[' ']] = 1
            

In [46]:
# Defining the input sequence and processing it
encoder_inputs = Input(shape = (None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard encoder_outputs and only keeping the hidden states(state_h) and the cell states(state_c)
encoder_states = [state_h, state_c]

In [47]:
# Setting up the decoder using 'encoder_states' as initial state
decoder_inputs = Input(shape = (None, num_decoder_tokens))
# We set up our decoder to return full output sequences and to return intenal states as well.
# We don't use the return states in the training model, but we'll be using them in the inference
decoder_lstm = LSTM(latent_dim, return_sequences = True,  return_state = True)
decoder_outputs, _,_ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [53]:
# Creating the model that will turn 'encoder_input_data' & 'decoder_input_data' into 'decoder_target_data'
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Training the model
model.compile(optimizer= 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
         batch_size = batch_size, epochs = epochs, validation_split = 0.4)

Epoch 1/100
94/94 [==============================] - 2s 22ms/step - loss: 0.0390 - accuracy: 0.9863 - val_loss: 0.3386 - val_accuracy: 0.9461
Epoch 2/100
94/94 [==============================] - 1s 14ms/step - loss: 0.0362 - accuracy: 0.9869 - val_loss: 0.3409 - val_accuracy: 0.9453
Epoch 3/100
94/94 [==============================] - 1s 15ms/step - loss: 0.0357 - accuracy: 0.9870 - val_loss: 0.3413 - val_accuracy: 0.9449
Epoch 4/100
94/94 [==============================] - 1s 15ms/step - loss: 0.0355 - accuracy: 0.9872 - val_loss: 0.3457 - val_accuracy: 0.9447
Epoch 5/100
94/94 [==============================] - 1s 14ms/step - loss: 0.0352 - accuracy: 0.9873 - val_loss: 0.3449 - val_accuracy: 0.9440
Epoch 6/100
94/94 [==============================] - 1s 14ms/step - loss: 0.0345 - accuracy: 0.9875 - val_loss: 0.3521 - val_accuracy: 0.9415
Epoch 7/100
94/94 [==============================] - 1s 14ms/step - loss: 0.0350 - accuracy: 0.9874 - val_loss: 0.3507 - val_accuracy: 0.9424
Epoch 

94/94 [==============================] - 1s 14ms/step - loss: 0.0284 - accuracy: 0.9880 - val_loss: 0.4764 - val_accuracy: 0.9052
Epoch 59/100
94/94 [==============================] - 1s 16ms/step - loss: 0.0284 - accuracy: 0.9879 - val_loss: 0.4674 - val_accuracy: 0.9072
Epoch 60/100
94/94 [==============================] - 1s 14ms/step - loss: 0.0283 - accuracy: 0.9881 - val_loss: 0.4673 - val_accuracy: 0.9060
Epoch 61/100
94/94 [==============================] - 1s 14ms/step - loss: 0.0283 - accuracy: 0.9878 - val_loss: 0.4698 - val_accuracy: 0.9060
Epoch 62/100
94/94 [==============================] - 1s 14ms/step - loss: 0.0280 - accuracy: 0.9881 - val_loss: 0.4730 - val_accuracy: 0.9057
Epoch 63/100
94/94 [==============================] - 1s 14ms/step - loss: 0.0283 - accuracy: 0.9881 - val_loss: 0.4759 - val_accuracy: 0.9049
Epoch 64/100
94/94 [==============================] - 1s 14ms/step - loss: 0.0282 - accuracy: 0.9881 - val_loss: 0.4742 - val_accuracy: 0.9049
Epoch 65/100

In [57]:
# Next: inference mode (sampling)
# Here's the drill:-
# 1. encode input and retrieve initial decoder state
# 2. run one step of decoder with this initial state and a 'start of sequence' token as target. Output will be the next target token.
# 3. repeat with the current target token and current states.

# Define sampling model
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape = (latent_dim,))
decoder_state_input_c = Input(shape = (latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state = decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, 
                     [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to something readable
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of training set) for trying out decoding
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input Sentence in English:', input_texts[seq_index])
    print('Decoded Sentence in Spanish:', decoded_sentence)

-
Input Sentence in English: Go.
Decoded Sentence in Spanish: Vete.

-
Input Sentence in English: Go.
Decoded Sentence in Spanish: Vete.

-
Input Sentence in English: Go.
Decoded Sentence in Spanish: Vete.

-
Input Sentence in English: Go.
Decoded Sentence in Spanish: Vete.

-
Input Sentence in English: Hi.
Decoded Sentence in Spanish: Hola.

-
Input Sentence in English: Run!
Decoded Sentence in Spanish: ¡Corre!

-
Input Sentence in English: Run!
Decoded Sentence in Spanish: ¡Corre!

-
Input Sentence in English: Run!
Decoded Sentence in Spanish: ¡Corre!

-
Input Sentence in English: Run!
Decoded Sentence in Spanish: ¡Corre!

-
Input Sentence in English: Run.
Decoded Sentence in Spanish: Corred.

-
Input Sentence in English: Who?
Decoded Sentence in Spanish: ¿Quién?

-
Input Sentence in English: Wow!
Decoded Sentence in Spanish: ¡Órale!

-
Input Sentence in English: Fire!
Decoded Sentence in Spanish: ¡Disparad!

-
Input Sentence in English: Fire!
Decoded Sentence in Spanish: ¡Disparad!
